# Vertex AI Multi-Agent Financial Research Assistant

Recreate the **Google Cloud × Schroders** multi-agent system to speed up company analysis from days to minutes.

_Last updated: 2025-07-01_


## 1. Setup and Configuration

In [ ]:
import os
from dotenv import load_dotenv
import vertexai

# Load environment variables from .env
load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION", "us-central1")
DATASTORE_ID = os.getenv("DATASTORE_ID")
BQ_DATASET = os.getenv("BQ_DATASET")
SEARCH_API_KEY = os.getenv("SEARCH_API_KEY")

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=REGION)
print(f"Vertex AI initialized for project {PROJECT_ID} in region {REGION}")


## 2. Define Financial Research Agents (ADK)

In [ ]:
from adk import Agent
from adk.tools import VertexAISearchTool, BigQueryTool, GoogleSearchTool

document_agent = Agent(
    name="document_search_agent",
    description="Answers queries from internal documents",
    model="gemini-1.5-flash",
    tools=[VertexAISearchTool(data_store_id=DATASTORE_ID)]
)

bigquery_agent = Agent(
    name="bigquery_agent",
    description="Handles structured data queries",
    model="gemini-1.5-flash",
    tools=[BigQueryTool(project_id=PROJECT_ID, dataset_id=BQ_DATASET)]
)

web_agent = Agent(
    name="web_search_agent",
    description="Searches the web for up-to-date market information",
    model="gemini-1.5-flash",
    tools=[GoogleSearchTool(api_key=SEARCH_API_KEY)]
)


### 2.1 Define Router Agent

In [ ]:
router_agent = Agent(
    name="router_agent",
    description="Routes queries to the correct agent based on task type",
    model="gemini-1.5-flash",
    tools=[document_agent, bigquery_agent, web_agent]
)


## 3. Orchestrate Agents with LangGraph

In [ ]:
from langgraph import Graph, Node

class RouterNode(Node):
    def process(self, text):
        if "10-K" in text or "risk" in text:
            return document_agent.process(text)
        elif "revenue" in text or "EPS" in text:
            return bigquery_agent.process(text)
        else:
            return web_agent.process(text)

graph = Graph(name="financial_research_assistant")
graph.add_node(RouterNode())
graph.add_node(document_agent)
graph.add_node(bigquery_agent)
graph.add_node(web_agent)
graph.compile()
print("LangGraph compiled.")


### 3.1 Run End-to-End Agent Workflow

In [ ]:
user_query = "Summarize Tesla's 10-K risks and compare with revenue from last year."
response = graph.run(user_query)
print(response)


## 4. Evaluate Performance with Vertex AI Evaluation

In [ ]:
from vertexai.preview.evaluation import Evaluation

evaluation = Evaluation(
    name="financial_research_eval",
    metrics=["accuracy", "latency", "tool_use"]
)

# Placeholder test data: update path to real JSONL with input-output pairs
# evaluation.run(router_agent, test_data="gs://your-bucket/test_data.jsonl")
print("Evaluation configuration completed. Add your test dataset to run.")


## 5. Deploy to Vertex AI Agent Engine

In [ ]:
from vertexai.preview import agent_engines

engine = agent_engines.create(agent_engine=router_agent)
print("Agent deployed:", engine.resource_name)


## 6. Cleanup Resources (Optional)

In [ ]:
# engine.delete()
# print("Agent Engine deleted.")
